<a href="https://colab.research.google.com/github/natskiu/Nasa-Battery/blob/main/Data_Visualisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#this is a comment
import scipy.io as sio

In [ ]:
url = 'https://github.com/natskiu/Nasa-Battery/blob/main/Data/B0005.mat?raw=true'
mat_file = sio.loadmat(url)

FileNotFoundError: ignored

In [ ]:
# raw=true is important so you download the file rather than the webpage.
!wget https://github.com/natskiu/Nasa-Battery/blob/main/Data/B0005.mat?raw=true
# rename the file
!mv B0005.mat\?raw\=true B0005.mat
# update scipy
!pip install -U -q scipy
# Load the data
from scipy import io
v = io.loadmat('B0005.mat')

--2021-08-16 07:31:55--  https://github.com/natskiu/Nasa-Battery/blob/main/Data/B0005.mat?raw=true
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/natskiu/Nasa-Battery/raw/main/Data/B0005.mat [following]
--2021-08-16 07:31:55--  https://github.com/natskiu/Nasa-Battery/raw/main/Data/B0005.mat
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/natskiu/Nasa-Battery/main/Data/B0005.mat [following]
--2021-08-16 07:31:56--  https://raw.githubusercontent.com/natskiu/Nasa-Battery/main/Data/B0005.mat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response.

In [ ]:
type(v)

dict

In [ ]:
v.keys()

dict_keys(['__header__', '__version__', '__globals__', 'B0005'])

In [ ]:
v['B0005'].shape

(1, 1)

In [41]:
type(v['B0005'])

numpy.ndarray

In [58]:
np.vstack(v['B0005'][0,0]).shape

(1, 616)

In [69]:
cycles = np.vstack(v['B0005'][0,0])
cycles.shape

(1, 616)

In [84]:
"only change the middle index i.e. cycles[0,1][3], cycles[0,2][3]"
cycles[0,0][3].shape

(1, 1)

In [88]:
"This is how you extract all the features from cycles"
np.vstack(cycles[0,0][3][0,0])[0]


array([3.87301722, 3.47939356, 4.00058782, 4.01239519, 4.01970806,
       4.02540947, 4.03063627, 4.03534896, 4.03971637, 4.04354121,
       4.04672407, 4.05032083, 4.05347776, 4.05687947, 4.06020401,
       4.06309148, 4.06606364, 4.06810568, 4.07091089, 4.07314062,
       4.07531198, 4.07798651, 4.07976022, 4.08180212, 4.08379322,
       4.08535888, 4.08707793, 4.0888699 , 4.08991841, 4.09204117,
       4.09327341, 4.09462341, 4.09636787, 4.09755211, 4.09852259,
       4.09990153, 4.10107397, 4.10186053, 4.10303834, 4.10433761,
       4.10518719, 4.10617751, 4.10713402, 4.108074  , 4.10876115,
       4.10971157, 4.11073785, 4.1115377 , 4.11216994, 4.11307997,
       4.11363282, 4.114555  , 4.11540239, 4.115793  , 4.11674636,
       4.117245  , 4.11794271, 4.1186426 , 4.11943651, 4.12004755,
       4.12062053, 4.12120233, 4.12214376, 4.12223722, 4.12330381,
       4.12376399, 4.12442084, 4.12507381, 4.12567826, 4.12616805,
       4.12656627, 4.12731238, 4.12808141, 4.12845744, 4.12895